In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import random_split,Dataset
from torch.nn import functional as F
import torchvision
from torchvision import datasets,transforms
import torchvision.transforms as transforms

from torch.distributions.normal import Normal

import os
import math
from tqdm import tqdm

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

from typing import List

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
#Script allow collab to get access to our shared Drive 
#If you are not he owner of the drive make sure you add ATML_HT22 shortcut to your drive or find other workaround 
import sys
from google.colab import drive
drive.mount('/content/drive')
sys.path.append('drive/MyDrive/ATML_HT22')

from train import train_and_test
from eval import CIFAR10

%load_ext autoreload
%autoreload 2

****

In [ ]:
# Parameters
batch_size          = 100

optim_lr            = 1e-5
rms_prop_momentum   = 0.9

num_training_steps  = 500000
imp_samples         = 200

# Model
D = 3*8*8 #784
encoder_hidden_dims = [400]
decoder_hidden_dims = [400]
latent_size         = 30
maxout_window_size  = 4
non_linearity       = "MaxOut" # "MaxOut" or "ReLU"
optim_type          = "RMSProp" # "RMSProp" or "Adam"

# Build Flows
flow_type           = 'Planar'
num_flow_blocks     = 0

# whether the dataset is binary
binary = False

settings = (batch_size, optim_lr, rms_prop_momentum, num_training_steps,\
            imp_samples, D, encoder_hidden_dims, decoder_hidden_dims, latent_size,\
            maxout_window_size, non_linearity, optim_type, flow_type, num_flow_blocks, binary)

____

In [ ]:
train_loader, test_loader = CIFAR10(batch_size)

In [ ]:
marg_lik, test_loss, train_losses, model = train_and_test("k0_CIFAR10", train_loader, test_loader, settings, DEVICE, do_not_load=True)

In [ ]:
from utils import load_model

#model, optimizer, steps, train_losses, settings = load_model("k2_CIFAR10", DEVICE)
#batch_size, optim_lr, rms_prop_momentum, num_training_steps,\
#imp_samples, D, encoder_hidden_dims, decoder_hidden_dims, latent_size,\
#maxout_window_size, non_linearity, optim_type, flow_type, num_flow_blocks, binary = settings

from train import test
from eval import estimate_marginal_likelihood

#test_loss = test(model, test_loader, binary, DEVICE)
#print(f"Final test loss: {test_loss:.2f}")
marg_log_lik = estimate_marginal_likelihood(20, test_loader, binary, model, DEVICE)
print(f"Marginal log likelihood: {marg_log_lik:.2f}")

In [ ]:
# random model
from models import NormalisingFlowModelVAE
model2 = NormalisingFlowModelVAE(dim_input = D,
                  e_hidden_dims = encoder_hidden_dims,
                  d_hidden_dims = decoder_hidden_dims,
                  flow_layers_num=num_flow_blocks,
                  non_linearity=non_linearity,
                  latent_size=latent_size,
                  maxout_window_size = maxout_window_size,
                  flow_type=flow_type,
                  ).to(DEVICE)
test_loss = test(model2, test_loader, binary, DEVICE)
print(f"Final test loss: {test_loss:.2f}")